In [1]:
# !gdown https://drive.google.com/file/d/1Hv4RAltBumSfOkRacoX8qrfDYfd_NDss/view?usp=drive_link --fuzzy

In [2]:
# !unzip Dataset_AML_Assignment1_Part1.zip

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
from IPython import display
from tqdm.notebook import tqdm
import random
import math, time, os
from matplotlib import pyplot as plt
import pickle as pkl


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
prefix = './data_dump_nal'


In [5]:
class SinusodialDataset(Dataset):
    def __init__(self, df, mode='era'):
        """ creating label columns of eras and targets """
        self.X = df.iloc[:, :24]
        if mode == 'era':
          self.y = df['era_label']
        else:
          self.y = df[f'{mode}']

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        X = torch.tensor(self.X.iloc[idx].values, dtype=torch.float32)
        y = torch.tensor(int(self.y.iloc[idx]), dtype=torch.long)
        return X, y

In [6]:
def make_data_splits(df, mode, batch_size=32, train_perc=0.7, val_test_perc=0.5):

    def encode(v, class_values):
        return class_values.index(v)

    #adding new era_label column indexed 0, 1,...
    class_values = df['era'].unique().tolist()
    df['era_label'] = df['era'].apply(lambda x: encode(x, class_values))
    df.reset_index(drop=True, inplace=True)

    train_samples = int(len(df)*train_perc)
    val_test_samples = len(df)-train_samples

    data = SinusodialDataset(df, mode=mode)
    data_train, data_test = random_split(data, [train_samples, val_test_samples])

    val_samples = int(len(data_test)*val_test_perc)
    test_samples = len(data_test)-val_samples
    data_val, data_test = random_split(data_test, [val_samples, test_samples])

    print("Train-val-test lengths: ", len(data_train), len(data_val), len(data_test))

    loader_train = DataLoader(data_train, batch_size=batch_size, shuffle=True)
    loader_val = DataLoader(data_val, batch_size=batch_size, shuffle=False)
    loader_test = DataLoader(data_test, batch_size=batch_size, shuffle=False)

    return loader_train, loader_val, loader_test, data

#NAL

In [127]:
class NAL_MLP(nn.Module):
    def __init__(self, dims, lr=1e-2, weight_decay=1e-3):
        super(NAL_MLP,self).__init__()
        self.dims=dims
        self.layers=nn.ModuleList()
        for i in range(len(self.dims)-2):
            self.layers.append(nn.Linear(dims[i],dims[i+1]))
            self.layers.append(nn.ReLU())

        self.output_layer = nn.Linear(dims[i+1],dims[i+2])
        self.softmax = nn.Softmax(dim=1)

        self.nal_head = nn.Linear(dims[i+1], 1)
        self.sigmoid = nn.Sigmoid()

        self.optimizer = optim.SGD(self.parameters(), lr=lr, weight_decay=weight_decay)

    def forward(self,x):
        x = x.float()
        for l in self.layers:
            x = l(x)
        output = self.softmax(self.output_layer(x))
        attn = self.sigmoid(self.nal_head(x))
        return output, attn

In [128]:
def accuracy(y_pred, y_test, verbose=True):
    m = y_test.shape[0]
    predicted = torch.max(y_pred, 1)[1]
    correct = (predicted == y_test).float().sum().item()
    if verbose:
        print(correct,m)
    accuracy = correct/m
    return accuracy, correct

In [129]:
def NAL_Loss(preds, attn, targets, lamb1=1, lamb2=1):
    #reduction sum
    # loss_a = -torch.sum( torch.log( preds[range(len(targets)), targets] ) )
    loss_a = -torch.sum(torch.log( torch.mul(attn, (preds[range(len(targets)), targets]-1)) + 1 ))
    loss_b = -torch.sum(torch.log(attn))
    return (lamb1*loss_a, lamb2*loss_b)
    # return loss_a


In [130]:
def Test(net, loader_test, mode, noise_level, device='cpu', Loss=NAL_Loss):
    net.eval()
    total_samples = 0
    correct_samples = 0
    loss = 0.0
    for (X, y) in loader_test:
        X=X.to(device)
        y=y.to(device)
        total_samples += y.shape[0]
        y_out, _ = net(X)
        _, i_cor_sam = accuracy(y_out,y,verbose=False)
        correct_samples += i_cor_sam
        y_out, y_attn = net(X)
        # loss += Loss(y_out, y).cpu().detach().item()
        loss_a, loss_b = Loss(y_out, y_attn, y)
        loss_a, loss_b = loss_a.cpu().detach().item(), loss_b.cpu().detach().item()
        loss += (loss_a+loss_b)
    acc = correct_samples / total_samples
    loss /= total_samples
    net.train()
    return loss, acc

In [131]:
def Train(Net, data, mode, noise_level, epochs=20, lr=5e-2, Loss=NAL_Loss, verbose=False, device='cpu',
          val_ds=None, plot_accs=False, plot_losses=False):
    model_save_time = time.time()
    losses = []
    accs = []
    val_losses=[]
    val_accL=[]
    Net.to(device)
    for e in range(epochs):
        Net.train()
        step=0
        tot_loss, tot_loss_a, tot_loss_b = 0.0, 0.0, 0.0
        start_time = time.time()
        correct_samples = 0
        total_samples = 0
        range_attn = [-100, 100] # max, min
        for (X,y) in data:
            X=X.to(device)
            y=y.to(device)
            total_samples += y.shape[0]
            y_out, y_attn = Net(X)

            if y_attn.min() < range_attn[1]:
                range_attn[1] = y_attn.min().item()
            if y_attn.max() > range_attn[0]:
                range_attn[0] = y_attn.max().item()

            loss_a, loss_b = Loss(y_out, y_attn, y)
            # loss = Loss(y_out, y)
            Net.optimizer.zero_grad()
            loss = loss_a+loss_b
            loss.backward()
            Net.optimizer.step()
            step+=1
            tot_loss_a+=loss_a
            tot_loss_b+=loss_b
            tot_loss+=loss
            if verbose:
                _, i_cor_sam = accuracy(y_out,y,verbose=False)
                correct_samples += i_cor_sam
        end_time = time.time()
        print("RANGE_ATTN:",range_attn)
        print("LOSS RATIO:",tot_loss_a.item(), tot_loss_b.item())
        t = end_time-start_time
        l = tot_loss.item()/total_samples
        losses += [l]
        if verbose:
            a = correct_samples/total_samples
            accs += [a]
            print('Epoch %2d Loss: %2.5e Accuracy: %2.5f Epoch Time: %2.5f' %(e,l,a,t))

        val_loss, val_acc = Test(Net, val_ds, mode, noise_level, device)
        val_losses.append(val_loss)
        val_accL.append(val_acc)

        torch.save(Net.state_dict(), f'{prefix}/net_{noise_level}_{mode}_{str(model_save_time)}.pth')

    return Net, losses, accs, val_losses, val_accL

In [132]:
def plot_loss_acc(losses, accs, val_losses, val_accs, mode, noise_level):

    plt.plot(np.array(accs),color='red', label='Train accuracy')
    plt.plot(np.array(val_accs),color='blue', label='Val accuracy')
    plt.legend()
    plt.savefig(f'{prefix}/acc_{mode}_{noise_level}.png')
    plt.clf()

    plt.plot(np.array(losses),color='red', label='Train loss')
    plt.plot(np.array(val_losses),color='blue', label='Val loss')
    plt.legend()
    plt.savefig(f'{prefix}/loss_{mode}_{noise_level}.png')
    plt.clf()
    return

In [133]:
df_paths = ['../Datasets/df_syn_train_0_0_.csv',
            '../Datasets/df_synA_train_shuffled.csv',
            '../Datasets/df_synA_test_hard_shuffled_sample.csv']

noise_levels = ['none', 'low', 'high']
batch_sizes = [8, 128, 128]

losses_arr = []
accs_arr = []

for i in range(0, 3):
    df = pd.read_csv(df_paths[i])

    modes = ['era', 'target_5_val', 'target_10_val']

    loss_per_mode = []
    acc_per_mode = []

    for mode in modes:
        print("Noise Level:", noise_levels[i], "Mode:", mode)
        loader_train, loader_val, loader_test, data = make_data_splits(df, mode=mode, \
                                                                       batch_size=batch_sizes[i])
        net = NAL_MLP(dims=[data.X.shape[1], 32, 64, 32, len(data.y.unique())]).to(device)
        net, losses, accs, val_losses, val_accL = Train(net, loader_train, mode, noise_levels[i], \
                                      epochs=50, verbose=True, device=device, val_ds=loader_val, \
                                      plot_accs=True, plot_losses=True)
        plot_loss_acc(losses, accs, val_losses, val_accL, mode, noise_levels[i])
        
        #Testing code
        test_loss, test_acc = Test(net, loader_test, mode, noise_levels[i], device=device)
        print(test_loss, test_acc)
        loss_per_mode.append(test_loss)
        acc_per_mode.append(test_acc)

    losses_arr.append(loss_per_mode)
    accs_arr.append(acc_per_mode)

Noise Level: none Mode: era
Train-val-test lengths:  5460 1170 1170
RANGE_ATTN: [0.5210817456245422, 0.09757984429597855]
LOSS RATIO: 5271.3857421875 11455.3125
Epoch  0 Loss: 3.06350e+00 Accuracy: 0.08828 Epoch Time: 2.25864
RANGE_ATTN: [0.13153067231178284, 0.11542418599128723]
LOSS RATIO: 5146.43505859375 11516.498046875
Epoch  1 Loss: 3.05182e+00 Accuracy: 0.09048 Epoch Time: 2.16335
RANGE_ATTN: [0.13165442645549774, 0.11673764884471893]
LOSS RATIO: 5145.76513671875 11515.0048828125
Epoch  2 Loss: 3.05143e+00 Accuracy: 0.09048 Epoch Time: 2.13929
RANGE_ATTN: [0.13237862288951874, 0.11672475188970566]
LOSS RATIO: 5145.12548828125 11511.90625
Epoch  3 Loss: 3.05074e+00 Accuracy: 0.12106 Epoch Time: 2.17624
RANGE_ATTN: [0.13202029466629028, 0.11572710424661636]
LOSS RATIO: 5142.50048828125 11501.6181640625
Epoch  4 Loss: 3.04837e+00 Accuracy: 0.09982 Epoch Time: 2.18165
RANGE_ATTN: [0.1662692278623581, 0.10948130488395691]
LOSS RATIO: 5129.44140625 11344.294921875
Epoch  5 Loss: 3.017

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [ ]:
with open(f'{prefix}/losses_dump.pkl', 'wb') as f:
    pkl.dump(losses_arr, f)

with open(f'{prefix}/accs_dump.pkl', 'wb') as f:
    pkl.dump(accs_arr, f)

In [ ]:
# import pickle as pkl
# prefix = './data_dump_NAL_final'
# with open(f'{prefix}/accs_dump.pkl', 'rb') as f:
#     accs = pkl.load(f)

# print(accs)

[[0.8905982905982905, 0.958974358974359, 0.9777777777777777], [0.7842948717948718, 0.9435042735042735, 0.9539957264957265], [0.5431089743589743, 0.8661057692307692, 0.8773771367521368]]
